In [6]:
import cv2
import numpy as np
import torch

from raft import RAFT  # Ensure RAFT is installed
from utils.utils import InputPadder  # Helper from RAFT's utilities
from argparse import Namespace

In [7]:
def read_file_and_extract_data(file_path):
    with open(file_path, 'r') as file:
        # Read the first three lines
        height = int(file.readline().strip())
        width = int(file.readline().strip())
        num_points = int(file.readline().strip())
        
        # Read the specified number of points
        data = []
        for _ in range(num_points):
            line = file.readline().strip()
            x, y = map(int, line.split(','))
            data.append((x, y))

    return height, width, num_points, data

In [8]:
file_path = '../../Lukas Kanade + Simulation/tmp/goodflower-fp.txt'
height, width, num_points, data_points = read_file_and_extract_data(file_path)

In [9]:
data_points=np.asarray(data_points)

In [ ]:
# Load pre-trained RAFT model
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
args = Namespace(
    small=True,
    mixed_precision=True,
    alternate_corr=False
)
raft_model = RAFT(args)
# Load the state_dict
state_dict = torch.load("../raft-kitti.pth",map_location=torch.device('cpu'))

# Remove 'module.' prefix from keys
new_state_dict = {k.replace('module.', ''): v for k, v in state_dict.items()}

# Load the modified state_dict into the model
raft_model.to(DEVICE)
raft_model.eval()



def calculate_optical_flow(prev_frame, next_frame, model):
    with torch.no_grad():
        # Prepare input
        prev_tensor = torch.from_numpy(prev_frame).permute(2, 0, 1).float().unsqueeze(0).to(DEVICE) / 255.0
        next_tensor = torch.from_numpy(next_frame).permute(2, 0, 1).float().unsqueeze(0).to(DEVICE) / 255.0
        padder = InputPadder(prev_tensor.shape)
        prev_tensor, next_tensor = padder.pad(prev_tensor, next_tensor)

        # Compute flow
        flow_low, flow_up = model(prev_tensor, next_tensor, iters=20, test_mode=True)
        return padder.unpad(flow_up[0].cpu().numpy())
    
    
# Helper function for RAFT optical flow
def compute_fft_components(optical_flow, fps, max_frequency=30, num_points=212):
    """
    Computes FFT components (real, imaginary) and magnitudes for optical flow up to a specified frequency.
    
    Args:
    - optical_flow: A 2D array with shape (H, W, 2), containing flow values for x and y directions.
    - fps: Frames per second of the input video.
    - max_frequency: Maximum frequency (Hz) to include in the FFT results.
    - num_points: Number of FFT points (limited to 451).
    
    Returns:
    - fft_dx_real, fft_dx_imag, fft_dy_real, fft_dy_imag: FFT components limited to max_frequency.
    - x_magnitudes, y_magnitudes: Magnitudes for dx and dy FFTs.
    - valid_frequencies: Frequencies up to max_frequency.

"""
    n_bins=int(num_points/2)
    h=optical_flow.shape[2]
    w=optical_flow.shape[3]
    fft_u = np.zeros((n_bins, h, w), dtype=complex)
    fft_v = np.zeros((n_bins, h, w), dtype=complex)


    for row in range(h):
        for col in range(w):
            pixel_fft_u = np.fft.fft(optical_flow[0, :, row, col], axis=0)
            pixel_fft_v = np.fft.fft(optical_flow[1, :, row, col], axis=0)

            fft_u[:, row, col] = pixel_fft_u[:n_bins]
            fft_v[:, row, col] = pixel_fft_v[:n_bins]

    # Compute frequency bins
    frequencies = np.fft.rfftfreq(num_points)*60
    frequencies = frequencies[:-1]


    return fft_u, fft_v, frequencies

def process_video_and_create_file(video_path, fps, max_frequency=30, num_points=212):
    cap = cv2.VideoCapture(video_path)
    ret, prev_frame = cap.read()

    if not ret:
        print("Failed to read video.")
        return

    optical_flow = np.zeros((2, num_points,prev_frame.shape[0] ,prev_frame.shape[1] ))

    i=0

    while True:
        ret, next_frame = cap.read()
        if not ret:
            break

        
        # Calculate optical flow using RAFT
        optical_flow[:,i,:,:]= calculate_optical_flow(prev_frame, next_frame, raft_model)
        i=+1
        prev_frame = next_frame
    # Compute FFT components (up to max_frequency)
    fft_u,fft_v, frequencies = compute_fft_components(optical_flow, fps, max_frequency, num_points)


    return fft_u, fft_v, frequencies


In [ ]:
fft_u, fft_v, frequencies=process_video_and_create_file("../../Lukas Kanade + Simulation/tmp/goodflower.mp4",30)
fft_u, fft_v, frequencies=process_video_and_create_file("../../Lukas Kanade + Simulation/tmp/goodflower.mp4",30)


In [13]:
def save_fft_python(fft_dx_real, fft_dx_imag, fft_dy_real, fft_dy_imag, freqs):
    # Remove the last frequency and corresponding data
    freqs = freqs[:-1]
    # fft_dx_real = fft_dx_real[:-1]
    # fft_dx_imag = fft_dx_imag[:-1]
    # fft_dy_real = fft_dy_real[:-1]
    # fft_dy_imag = fft_dy_imag[:-1]

    # Create output file
    output_name = f"goodflower-fft.txt"
    
    with open(output_name, "w") as fs:
        n_freqs = len(freqs)
        n_features = fft_dx_real.shape[1]
        
        # Write number of frequencies and features
        fs.write(f"{n_freqs}\n")
        fs.write(f"{n_features}\n")
        
        # Calculate magnitudes and write them along with frequencies
        for i in range(n_freqs):
            x_mags = np.sqrt(fft_dx_real[i]**2 + fft_dx_imag[i]**2)
            y_mags = np.sqrt(fft_dy_real[i]**2 + fft_dy_imag[i]**2)
            x_mag_sum = np.sum(x_mags)
            y_mag_sum = np.sum(y_mags)
            fs.write(f"{freqs[i]},{x_mag_sum},{y_mag_sum}\n")
        
        # Helper function to write matrix data
        def write_cv_mat(mat):
            for row in mat:
                fs.write(",".join(map(str, row)) + "\n")
        
        # Write matrix data
        write_cv_mat(fft_dx_real)
        write_cv_mat(fft_dx_imag)
        write_cv_mat(fft_dy_real)
        write_cv_mat(fft_dy_imag)

    print(f"File saved as {output_name}")

In [14]:
save_fft_python(np.real(fft_u[:,data_points[:,1],data_points[:,0]]), np.imag(fft_u[:,data_points[:,1],data_points[:,0]]), np.real(fft_v[:,data_points[:,1],data_points[:,0]]), np.imag(fft_v[:,data_points[:,1],data_points[:,0]]), frequencies)

File saved as goodflower-fft.txt
